In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

In [3]:
# 載入和預處理數據
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# 填補空缺數據和數據編碼
train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
test_df['Age'].fillna(test_df['Age'].median(), inplace=True)
test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
test_df['Embarked'].fillna(test_df['Embarked'].mode()[0], inplace=True)

train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1})
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1})
train_df['Embarked'] = train_df['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})
test_df['Embarked'] = test_df['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

# 選擇特徵和標籤
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = train_df[features].values
y = train_df['Survived'].values
X_test = test_df[features].values

# 數據標準化
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

C:\Users\user\AppData\Local\Temp\ipykernel_14512\3809197126.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14512\3809197126.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

In [4]:
# 分割訓練和驗證集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 構建神經網絡模型
model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 編譯模型
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# 訓練模型
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32)

Epoch 1/50
23/23 [==============================] - 1s 7ms/step - loss: 0.6219 - accuracy: 0.7079 - val_loss: 0.5445 - val_accuracy: 0.7989
Epoch 2/50
23/23 [==============================] - 0s 3ms/step - loss: 0.4927 - accuracy: 0.8160 - val_loss: 0.4529 - val_accuracy: 0.7989
Epoch 3/50
23/23 [==============================] - 0s 3ms/step - loss: 0.4445 - accuracy: 0.8090 - val_loss: 0.4234 - val_accuracy: 0.7989
Epoch 4/50
23/23 [==============================] - 0s 4ms/step - loss: 0.4228 - accuracy: 0.8174 - val_loss: 0.4199 - val_accuracy: 0.8101
Epoch 5/50
23/23 [==============================] - 0s 3ms/step - loss: 0.4095 - accuracy: 0.8287 - val_loss: 0.4217 - val_accuracy: 0.8156
Epoch 6/50
23/23 [==============================] - 0s 3ms/step - loss: 0.4068 - accuracy: 0.8357 - val_loss: 0.4288 - val_accuracy: 0.8101
Epoch 7/50
23/23 [==============================] - 0s 3ms/step - loss: 0.4007 - accuracy: 0.8343 - val_loss: 0.4346 - val_accuracy: 0.8045
Epoch 8/50
23/23 [==

In [5]:
# 預測驗證集並計算混淆矩陣和準確度
y_pred_val = (model.predict(X_val) > 0.5).astype("int32")
conf_matrix = confusion_matrix(y_val, y_pred_val)
accuracy = accuracy_score(y_val, y_pred_val)

print("Confusion Matrix:")
print(conf_matrix)
print("\nAccuracy:", accuracy)

Confusion Matrix:
[[95 10]
 [23 51]]

Accuracy: 0.8156424581005587


In [6]:
# 對測試集進行預測
y_test_pred = (model.predict(X_test) > 0.5).astype("int32")

# 輸出預測結果至 CSV 文件
output = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': y_test_pred.flatten()})
output.to_csv("output.csv", index=False)
print("預測結果已保存至 output.csv")

預測結果已保存至 output.csv
